# 双样本比例z检验（Two-Proportion Z-Test）

## 基本概念
**双样本比例Z检验**用于比较两个独立样本的比例是否存在显著差异

## 适用场景
- 两组样本相互独立（来自两个不同总体）
- 样本量较大，满足正态近似（通常每组n ≥ 30）

## 零假设与备择假设
- 单尾检验：
    - 右尾检验：
        - 零假设（$H_0$）：第一组总体比例小于等于第二组总体比例，即 $p_1 <= p_2$。
        - 备择假设（$H_1$）：第一组总体比例大于第二组总体比例，即$p_1 > p_2$。
    - 左尾检验：
        - 零假设（$H_0$）：第一组总体比例大于等于第二组总体比例，即 $p_1 >= p_2$。
        - 备择假设（$H_1$）：第一组总体比例小于第二组总体比例，即$p_1 < p_2$。
- 双尾检验：
    - 零假设（$H_0$）：两组数据总体比例相等，即 $p_1 = p_2$。
    - 备择假设（$H_1$）：两组数据总体比例不相等，即$p_1 \neq p_2$。

## 检验统计量
检验统计量 $Z$ 的计算公式为：
$$Z = \frac{(\hat{p}_1 - \hat{p}_2) - 0}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$
其中：
- $\hat{p}_1$, $\hat{p}_2$：两样本比例（如转化率）
- $n_1$, $n_2$：两样本容量
- $\hat{p}$：合并比例（当H₀为真时两组的共同比例估计）： 
    - $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$，$x_1$, $x_2$为两组的成功次数

## 步骤
- **提出假设**：
    - 明确原假设（$H_0$）和备择假设（$H_1$）
- **计算Z值**：
    - 使用样本数据计算出$Z$值。 
- **查找临界值**：
    - 根据显著性水平$\alpha$，查找标准正态分布表中的临界值$Z_{\text{临界}}$。 
        - 单尾检验：临界值为 $Z_{\text{临界}} = Z_{\alpha}$。
        - 双尾检验：临界值为 $Z_{\text{临界}} = Z_{\alpha/2}$。
- **做出决策**：
    - 如果$|Z| > Z_{\text{临界}}$，拒绝零假设。
    - 如果$|Z| \leq Z_{\text{临界}}$，不能拒绝零假设。

# 双样本比例z检验实际案例

## 单尾检验

### 案例背景
某电商平台对首页促销 banner 进行了两种设计（A组：静态图片，B组：动态轮播），希望验证动态设计的点击率（CTR）是否显著高于静态设计（左尾检验）。
- 实验数据：
    - A组（静态）：曝光量=10,000次，点击次数=450次（CTR=4.5%）
    - B组（动态）：曝光量=10,000次，点击次数=600次（CTR=6.0%）

### 确定假设
- **零假设（$H_0$）**：静态设计CTR >= 动态设计，即$p_1 >= p_2$
- **备择假设（$H_1$）**：静态设计CTR < 动态设计，即$p_1 < p_2$

### 计算检验统计量
$$Z = \frac{(\hat{p}_1 - \hat{p}_2) - 0}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$
其中：
- $\hat{p}_1$, $\hat{p}_2$：两样本比例（如转化率）
- $n_1$, $n_2$：两样本容量
- $\hat{p}$：合并比例（当H₀为真时两组的共同比例估计）： 
    - $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$，$x_1$, $x_2$为两组的成功次数

### 代码实现

#### 手工计算

In [5]:
import numpy as np
from scipy.stats import norm

# 输入数据
x1, n1 = 450, 10000  # A组点击次数和曝光量
x2, n2 = 600, 10000  # B组点击次数和曝光量

# 计算样本比例和合并比例
p1, p2 = x1/n1, x2/n2
p_pool = (x1 + x2) / (n1 + n2)

# 计算Z统计量（左尾检验）
z_score = (p1 - p2) / np.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2))
p_value = 1 - norm.sf(z_score)  # 左尾P值

# 输出结果
print(f"A组CTR: {p1:.2%}  B组CTR: {p2:.2%}")
print(f"Z统计量: {z_score:.4f}")
print(f"P值: {p_value:.4f}")

# 显著性判断（α=0.05）
alpha = 0.05
if p_value < alpha:
    print("结论：动态设计CTR显著更高（拒绝H₀）")
else:
    print("结论：无显著提升（无法拒绝H₀）")

A组CTR: 4.50%  B组CTR: 6.00%
Z统计量: -4.7556
P值: 0.0000
结论：动态设计CTR显著更高（拒绝H₀）


#### `proportions_ztest()`实现

`proportions_ztest()`
- **输入参数**
   - `count`: 成功次数
       - 单样本检验: 直接传入成功次数
       - 双样本检验: 传入数组$[count1, count2]$，分别代表两组样本的成功次数
   - `nobs`: 总实验次数
      - 单样本检验: 传入总试验次数
      - 双样本检验: 传入数组$[nobs1, nobs2]$，分别代表两组样本的总试验次数
   - `value`: 原假设的比例值或比例差异值
       - 单样本检验：原假设的比例
       - 双样本检验：两组比例的差异值
   - `alternative`: 备择假设的类型
       - `two-sided`：双尾检验（默认）
       - `larger`：右尾检验（备择假设中检验参数是否显著大于某个值）
       - `smaller`：左尾检验（备择假设中检验参数是否显著小于某个值）
   - `prop_var`: 标准误差的计算方式
       - `False (默认)`: 使用样本比例$\hat{p}$计算标准误差
       - 指定比例（如 0.05）: 使用指定值计算标准误差，一般使用原假设比例$p_0$
- **输出返回值**
    - `zstat`：Z 统计量（检验统计量）。
    - `pvalue`：P 值

In [9]:
from statsmodels.stats.proportion import proportions_ztest

# 输入点击次数和曝光量
count = np.array([450, 600])    # [A组点击, B组点击]
nobs = np.array([10000, 10000]) # 两组曝光量

# 单尾Z检验（左尾：alternative='smaller'）
z_stat, p_value = proportions_ztest(count, nobs, value=0, alternative='smaller', prop_var=0)

print(f"Z统计量: {z_stat:.4f}")
print(f"P值: {p_value:.4f}")

if p_value < 0.05:
    print("结论：动态设计显著提升CTR（拒绝H₀）")
else:
    print("结论：无显著差异")

Z统计量: -4.7556
P值: 0.0000
结论：动态设计显著提升CTR（拒绝H₀）


## 双尾检验

### 案例背景
某互联网广告平台对广告落地页进行了两种设计（A组：单列布局，B组：双列布局），希望验证两种设计的用户注册转化率是否存在显著差异（双尾检验）。

- 实验数据：
    - A组（单列）：曝光量=8,000次，注册量=320人（转化率=4.0%）
    - B组（双列）：曝光量=8,000次，注册量=400人（转化率=5.0%）

### 确定假设
- **零假设（$H_0$）**：两设计转化率无差异，即$p_A = p_B$
- **备择假设（$H_1$）**：两设计转化率存在差异，即$p_A \ne p_B$

### 计算检验统计量
$$Z = \frac{(\hat{p}_1 - \hat{p}_2) - 0}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$
其中：
- $\hat{p}_1$, $\hat{p}_2$：两样本比例（如转化率）
- $n_1$, $n_2$：两样本容量
- $\hat{p}$：合并比例（当H₀为真时两组的共同比例估计）： 
    - $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$，$x_1$, $x_2$为两组的成功次数

### 代码实现

#### 手工计算

In [10]:
import numpy as np
from scipy.stats import norm

# 输入数据
x1, n1 = 320, 8000   # A组注册量和曝光量
x2, n2 = 400, 8000   # B组注册量和曝光量

# 计算样本比例和合并比例
p1, p2 = x1/n1, x2/n2
p_pool = (x1 + x2) / (n1 + n2)

# 计算Z统计量（双尾检验）
z_score = (p1 - p2) / np.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2))
p_value = 2 * norm.sf(np.abs(z_score))  # 双尾需乘以2

# 输出结果
print(f"A组转化率: {p1:.2%}  B组转化率: {p2:.2%}")
print(f"Z统计量: {z_score:.4f}")
print(f"P值: {p_value:.4f}")

# 显著性判断（α=0.05）
alpha = 0.05
if p_value < alpha:
    print("结论：两种设计转化率差异显著（拒绝H₀）")
else:
    print("结论：无显著差异（无法拒绝H₀）")

A组转化率: 4.00%  B组转化率: 5.00%
Z统计量: -3.0509
P值: 0.0023
结论：两种设计转化率差异显著（拒绝H₀）


#### `proportions_ztest()`实现

In [12]:
from statsmodels.stats.proportion import proportions_ztest

# 输入点击次数和曝光量
count = np.array([320, 400])    # [A组注册量, B组注册量]
nobs = np.array([8000, 8000])   # 两组曝光量

# 双尾Z检验（默认alternative='two-sided'）
z_stat, p_value = proportions_ztest(count, nobs, value=0, alternative='two-sided', prop_var=0)

print(f"Z统计量: {z_stat:.4f}")
print(f"P值: {p_value:.4f}")

if p_value < 0.05:
    print("结论：转化率差异显著（拒绝H₀）")
else:
    print("结论：无显著差异")

Z统计量: -3.0509
P值: 0.0023
结论：转化率差异显著（拒绝H₀）
